In [5]:
import numpy as np
# pandas
from pandas import read_csv
# matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
#teras
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.metrics import AUC, Accuracy, Precision, Recall, FalsePositives

# sklearn
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold

In [6]:
seed = 7
np.random.seed(seed)



In [ ]:
# load dataset
dataframe = read_csv("analysis.csv", header=0)
dataset = dataframe.values
#dataset = np.random.shuffle(dataset)
X = dataset[:,1:14].astype(float)   # columns from 2nd to 13th into X
Y = dataset[:,14] #label column (14th) into Y 

In [ ]:
#Encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)

#One-hot encoding
transformed_Y = to_categorical(encoded_Y)

In [ ]:
model = Sequential()
model.add(Dense(5, activation='relu', input_shape=(13,)))
#model.add(Dropout(0.5))
model.add(Dense(5, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))


model.compile(loss='binary_crossentropy', 
              optimizer='adam',
              metrics=[precision(), recall()])

X_train, X_test, Y_train, Y_test = train_test_split(X,
                                                    transformed_Y,
                                                    test_size=0.2,
                                                    random_state=seed,
                                                    shuffle = True)
history=model.fit(X_train,
                  Y_train,
                  epochs=15,
                  validation_data=(X_test,Y_test),
                  batch_size=500)